## Granger Causality Test

Companies that Granger Causes Apple’s close differencing will be implemented in VAR.

In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
from statsmodels.tsa.stattools import grangercausalitytests

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## Function that export p-values for Granger Causality Test

In [3]:
## https://www.machinelearningplus.com/time-series/granger-causality-test-in-python/
def grangers_causation_matrix(data, variables,test='ssr_chi2test', verbose=False):    
    """Check Granger Causality of all possible combinations of the Time series.
    The rows are the response variable, columns are predictors. The values in the table 
    are the P-Values. P-Values lesser than the significance level (0.05), implies 
    the Null Hypothesis that the coefficients of the corresponding past values is 
    zero, that is, the X does not cause Y can be rejected.

    data      : pandas dataframe containing the time series variables
    variables : list containing names of the time series variables.
    """
    df = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    for c in df.columns:
        for r in df.index:
            test_result = grangercausalitytests(data[[r, c]], maxlag=10, verbose=False)
            p_values = [round(test_result[i+1][0][test][1],4) for i in range(10)]
            if verbose: print(f'Y = {r}, X = {c}, P Values = {p_values}')
            min_p_value = np.min(p_values)
            #opt_lag = 1 + np.argmin(p_values)
            df.loc[r, c] = min_p_value
    df.columns = [var + '_x' for var in variables]
    df.index = [var + '_y' for var in variables]
    return df

## Import stock data from Yahoo Finance

In [4]:
google = yf.Ticker("GOOG")
apple = yf.Ticker("AAPL")
microsoft = yf.Ticker("MSFT")
nvidia = yf.Ticker("NVDA")
amazon = yf.Ticker("AMZN")
meta = yf.Ticker("META")
tsmc = yf.Ticker("TSM")

google_data = google.history(start='2023-01-01', end='2024-01-01')
apple_data = apple.history(start='2023-01-01', end='2024-01-01')
microsoft_data = microsoft.history(start='2023-01-01', end='2024-01-01')
nvidia_data = nvidia.history(start='2023-01-01', end='2024-01-01')
amazon_data = amazon.history(start='2023-01-01', end='2024-01-01')
meta_data = meta.history(start='2023-01-01', end='2024-01-01')
tsm_data = tsmc.history(start='2023-01-01', end='2024-01-01')

## Create Dataframe for Granger Causality test

In [5]:
stationary_df = pd.DataFrame()
stationary_df["Apple Close Diff"] = apple_data.Close.diff()
stationary_df["Google Close Diff"] = google_data.Close.diff()
stationary_df["Microsoft Close Diff"] = microsoft_data.Close.diff()
stationary_df["Nvidia Close Diff"] = nvidia_data.Close.diff()
stationary_df["Amazon Close Diff"] = amazon_data.Close.diff()
stationary_df["Meta Close Diff"] = meta_data.Close.diff()
stationary_df["TSMC Close Diff"] = meta_data.Close.diff()

## Drop the first NA row
stationary_df = stationary_df.dropna()

In [6]:
grangers_causation_matrix(stationary_df, variables=stationary_df.columns)

,Apple Close Diff_x,Google Close Diff_x,Microsoft Close Diff_x,Nvidia Close Diff_x,Amazon Close Diff_x,Meta Close Diff_x,TSMC Close Diff_x
Apple Close Diff_y,1.0000,0.5876,0.4740,0.0158,0.4873,0.5645,0.5645
Google Close Diff_y,0.0713,1.0000,0.2047,0.2592,0.1273,0.0957,0.0957
Microsoft Close Diff_y,0.1333,0.3904,1.0000,0.2939,0.5907,0.5403,0.5403
Nvidia Close Diff_y,0.6800,0.8959,0.2444,1.0000,0.3770,0.4903,0.4903
Amazon Close Diff_y,0.0023,0.0064,0.1589,0.0837,1.0000,0.0059,0.0059
Meta Close Diff_y,0.0194,0.0369,0.3946,0.2334,0.3620,1.0000,1.0000
TSMC Close Diff_y,0.0194,0.0369,0.3946,0.2334,0.3620,1.0000,1.0000


From this table, we found that the NVIDIA's close difference Granger Causes Apple's Close Difference NVIDIA's close difference (p = 0.0158).